# Submitting calculations
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO (PW) calculation
Time: 3 mins

In [24]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from ase.lattice.spacegroup import crystal

import nglview
#import warnings
#warnings.filterwarnings('ignore')

#### Create a diamond cubic crystal structure

In [25]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

<StructureData: uuid: 2dd9f48b-115c-46ec-aba3-35aee1b15849 (pk: 5360)>

#### Show the already stored structure

In [26]:
view = nglview.show_ase(structure.get_ase())
view

The installed widget Javascript is the wrong version.


#### Create the _k_-points mesh

In [27]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

<KpointsData: uuid: 5e37c0dc-a398-4067-a56a-500425e91ad1 (pk: 5361)>

#### List existing pseudopotential families

In [28]:
!verdi data upf listfamilies

* GBRV_pbe [33 pseudos]
* GBRV_lda [33 pseudos]
* GBRV_pbesol [33 pseudos]
* SSSP [85 pseudos]


#### Define the calculation input parameters

In [29]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### See the available codes and select one of them

In [30]:
!verdi code list

# List of configured codes:
# (use 'verdi code show CODEID' to see the details)
* pk 4681 - pw-5.1@localhost


#### Get the Code instance from the name

In [31]:
codename = 'pw-5.1@localhost'
code = Code.get_from_string(codename)

#### Create a new calculation from the code and set the input parameters

In [32]:
calc = code.new_calc()
calc.use_structure(structure)
calc.use_pseudos_from_family(pseudo_family)
calc.use_parameters(ParameterData(dict=parameters))
calc.use_kpoints(kpoints)
calc.set_resources({"num_machines": 1})
calc.set_max_wallclock_seconds(30*60)

#### Store the calculation and submit it to the daemon

In [33]:
calc.store_all()
calc.submit()
print "Submitted a {} with pk<{}>".format(calc.__class__.__name__, calc.pk)

Submitted a PwCalculation with pk<5363>


#### Check the status of the calculation

In [34]:
!verdi calculation list

# Last daemon state_updater check: 0h:00m:01s ago (at 17:22:58 on 2017-04-24)
  PK  State          Creation    Sched. state    Computer    Type
----  -------------  ----------  --------------  ----------  ------------------
5363  WITHSCHEDULER  6s ago      RUNNING         localhost   quantumespresso.pw

Total results: 1



#### Monitor its status until completion

In [35]:
import time
from IPython.display import clear_output

def check_calculation_status(calc):
    while not calc.has_finished():
        clear_output()
        !verdi calculation list
        time.sleep(2)

    clear_output()
    !verdi calculation list $calc.pk

In [36]:
check_calculation_status(calc)

# Last daemon state_updater check: 0h:00m:00s ago (at 17:23:12 on 2017-04-24)
  PK  State     Creation    Sched. state    Computer    Type
----  --------  ----------  --------------  ----------  ------------------
5363  FINISHED  20s ago     DONE            localhost   quantumespresso.pw

Total results: 1



#### Check that the calculation finished successfully

In [37]:
calc.has_finished_ok()

True

#### We can easily show all the attached outputs

In [38]:
calc.get_outputs_dict()

{u'output_array': <ArrayData: uuid: bd00645f-f7d9-4a36-a33c-d75d39e71aa1 (pk: 5368)>,
 u'output_array_5368': <ArrayData: uuid: bd00645f-f7d9-4a36-a33c-d75d39e71aa1 (pk: 5368)>,
 u'output_kpoints': <KpointsData: uuid: 938921d8-1e0f-4b31-8bae-6081544703d0 (pk: 5366)>,
 u'output_kpoints_5366': <KpointsData: uuid: 938921d8-1e0f-4b31-8bae-6081544703d0 (pk: 5366)>,
 u'output_parameters': <ParameterData: uuid: a938786d-14c1-4fc4-b223-34325f1419b2 (pk: 5367)>,
 u'output_parameters_5367': <ParameterData: uuid: a938786d-14c1-4fc4-b223-34325f1419b2 (pk: 5367)>,
 u'remote_folder': <RemoteData: uuid: 2b84f89a-1ae2-4b4f-bbd4-b0add67deaa2 (pk: 5364)>,
 u'remote_folder_5364': <RemoteData: uuid: 2b84f89a-1ae2-4b4f-bbd4-b0add67deaa2 (pk: 5364)>,
 u'retrieved': <FolderData: uuid: 8034ad4b-ac3a-4c0c-855f-97b4355fb44d (pk: 5365)>,
 u'retrieved_5365': <FolderData: uuid: 8034ad4b-ac3a-4c0c-855f-97b4355fb44d (pk: 5365)>}

#### There are convenient methods to directly access the results

In [39]:
print "Total energy = {} {}".format(calc.res.energy, calc.res.energy_units)

Total energy = -2471.82841453 eV
